In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup 
import requests

from RGGI_plant_analysis import RGGI_capacity

In [ ]:
next_update_time,recent_report,report_month,report_year,date_of_last_report  =RGGI_capacity().scrape_recent_EIA_860m(lagged_report=1)

In [ ]:
plants = pd.read_excel(recent_report,sheet_name='Operating')
header_col = plants[(plants.iloc[:,0]=='Entity ID')==True].index[0]
plants = pd.read_excel(recent_report,sheet_name='Operating',header=header_col+1)
plants['report_month'] = report_month
plants['report_year'] = report_year
plants = plants.dropna(subset=['Plant Name'])
PJM_plants =  plants.query(f'`Balancing Authority Code`=="PJM" ')
full_tech_list = plants.Technology.unique()

PJM_plants = RGGI_capacity().analyse_RGGI_capacity(PJM_plants)

In [ ]:
planned = pd.read_excel(recent_report,sheet_name='Planned',header=2)
PJM_planned = planned.query(f'`Balancing Authority Code`=="PJM"')
PJM_planned['rggi_state'] = np.where(PJM_planned['Plant State'].isin(RGGI_capacity().RGGI_states),1,0)
RGGI_PJM_planned = PJM_planned.query('rggi_state==1 and `Nameplate Capacity (MW)`>25') 
additions,approved_additions,not_yet_approved_additions = RGGI_capacity().analyse_RGGI_planned_capacity(PJM_planned)
pjm_rggi_additions,pjm_rggi_approved_additions,pjm_rggi_not_yet_approved_additions = RGGI_capacity().analyse_RGGI_planned_capacity(RGGI_PJM_planned)

In [ ]:
planned_fossil_retirments_fig,all_planned_capacity_fig,approved_capacity_fig, not_yet_approved_capacity_fig = RGGI_capacity().RGGI_capacity_charts(PJM_plants,additions,approved_additions,not_yet_approved_additions)

In [ ]:
planned_fossil_retirments_fig

In [ ]:
all_planned_capacity_fig

In [ ]:
approved_capacity_fig

In [ ]:
not_yet_approved_capacity_fig

In [ ]:
PJM_plants['rggi_state'] = np.where(PJM_plants['Plant State'].isin(RGGI_capacity().RGGI_states),1,0)

PJM_RGGI_plants = PJM_plants.query('rggi_state==1 and `Nameplate Capacity (MW)`>25')

In [ ]:
rggi_plus_retirements_plus_additions_timeseries, time_series_capacity_fig = RGGI_capacity().estimated_timeseries_capacity(PJM_plants,full_tech_list,additions,date_of_last_report,PJM_retiredates=True)

In [ ]:
rggi_pjm_rggi_plus_retirements_plus_additions_timeseries, rggi_pjm_time_series_capacity_fig = RGGI_capacity().estimated_timeseries_capacity(PJM_RGGI_plants,full_tech_list,pjm_rggi_additions,date_of_last_report,PJM_retiredates=True)

In [ ]:
rggi_share_pjm = rggi_plus_retirements_plus_additions_timeseries.stack().to_frame('PJM').join(rggi_pjm_rggi_plus_retirements_plus_additions_timeseries.stack().to_frame('RGGI_PJM'))

In [ ]:
tech_convert_dict = {'Petroleum Liquids':'Oil',
                    'Onshore Wind Turbine':'Wind',
       'Conventional Hydroelectric':'Hydro', 
       'Conventional Steam Coal':'Coal',
       'Natural Gas Fired Combined Cycle':'Gas', 
       'Natural Gas Steam Turbine':'Gas',
       'Natural Gas Fired Combustion Turbine':'Gas', 
       'Nuclear':'Nuclear',
       'Hydroelectric Pumped Storage':'Storage',
       'Natural Gas Internal Combustion Engine':'Gas', 
       'Batteries':'Storage',
       'Solar Photovoltaic':'Solar', 
       'Geothermal':'Other Renewables', 
       'Wood/Wood Waste Biomass':'Other Renewables',
       'Coal Integrated Gasification Combined Cycle':'Gas',
        'Other Gases':'Gas',
       'Petroleum Coke':'Coal', 
       'Municipal Solid Waste':'Other', 
       'Landfill Gas':'Other Renewables',
       'Natural Gas with Compressed Air Storage':'Gas', 
       'All Other':'Other',
       'Other Waste Biomass':'Other Renewables', 
       'Solar Thermal without Energy Storage':'Solar',
       'Other Natural Gas':'Gas', 
       'Solar Thermal with Energy Storage':'Solar',
       'Flywheels':'Other', 
       'Offshore Wind Turbine':'Wind', 
       'Hydrokinetic':'Hydro',
       'Other Energy Storage':'Storage'}


Fossil_tech = ['Coal','Gas','Oil']

In [ ]:
rggi_share_pjm = rggi_share_pjm.reset_index()#.loc['']

In [ ]:
rggi_share_pjm

In [ ]:
rggi_share_pjm['PJM_tech'] = rggi_share_pjm['level_1'].map(tech_convert_dict)

In [ ]:
rggi_share_pjm = rggi_share_pjm.groupby(['level_0','PJM_tech'])[['PJM','RGGI_PJM']].sum()

In [ ]:
rggi_share_pjm['RGGI_share_PJM'] = rggi_share_pjm['RGGI_PJM']/rggi_share_pjm['PJM']
rggi_share_pjm['RGGI_share_PJM'] = rggi_share_pjm['RGGI_share_PJM'].fillna(0)

In [ ]:
rggi_share_pjm

In [ ]:
px.line(rggi_share_pjm.reset_index(),x='level_0',y='RGGI_share_PJM',color='PJM_tech')

In [ ]:
px.line(rggi_share_pjm.loc[:,'Gas',:]['RGGI_share_PJM'])

In [ ]:
time_series_capacity_fig.update_layout(width=1000)

In [ ]:
rggi_pjm_time_series_capacity_fig